<a href="https://colab.research.google.com/github/jcjcchang/2nd-ML100Days/blob/master/homework/Day_019_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [82]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression


from google.colab import drive
drive.mount('gdrive')

df_train = pd.read_csv('gdrive/My Drive/draft-ds/titanic_train.csv')
df_test = pd.read_csv('gdrive/My Drive/draft-ds/titanic_test.csv')
df_train.shape
'''
data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')
'''
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()


Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [83]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [84]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [85]:
df.isnull().sum().sort_values(ascending=False)

Age       263
Fare        1
Parch       0
SibSp       0
Pclass      0
dtype: int64

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [87]:
# 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
train_X = df_m1[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())



0.6960299128976762


In [91]:
# 空值補0, 做羅吉斯迴歸
df_0 = df.fillna(0)
train_X = df_0[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())

0.6971535084032942


In [97]:
# 空值補Mean, 做羅吉斯迴歸
df_mn = df.fillna(df.Age.mean())
train_X = df_mn[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())

0.6981761033723469


In [98]:
#這個比較正確 !! 
# 空值補Mean2, 做羅吉斯迴歸
df_mn2 = df.fillna(df.mean())
train_X = df_mn2[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())

0.6981761033723469


In [109]:
# 搭配標準化
df_temp = StandardScaler().fit_transform(df)
train_X = df_mn2[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())

0.6981761033723469


In [117]:
# 空值補 -1, 搭配最大最小化
df_maxmin = df.fillna(-1)
df_maxmin = MinMaxScaler().fit_transform(df)
train_X = df_maxmin[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

ValueError: ignored

In [104]:
df_mn2.head(154)

,Pclass,Age,SibSp,Parch,Fare
0,3,22.000000,1,0,7.2500
1,1,38.000000,1,0,71.2833
2,3,26.000000,0,0,7.9250
3,1,35.000000,1,0,53.1000
4,3,35.000000,0,0,8.0500
5,3,29.881138,0,0,8.4583
6,1,54.000000,0,0,51.8625
7,3,2.000000,3,1,21.0750
8,3,27.000000,0,2,11.1333
9,2,14.000000,1,0,30.0708


In [103]:
df.mean()

Pclass     2.294882
Age       29.881138
SibSp      0.498854
Parch      0.385027
Fare      33.295479
dtype: float64

In [0]:
nan_index = df[df.Age.isna()].reset_index()['index']
nan_index

In [108]:
df_mn2[df.Fare.isna()]

,Pclass,Age,SibSp,Parch,Fare
152,3,60.5,0,0,33.295479


In [33]:
#df_0 = df.fillna(0)
df[df.Age.isna()]

,Pclass,Age,SibSp,Parch,Fare
5,3,NaN,0,0,8.4583
17,2,NaN,0,0,13.0000
19,3,NaN,0,0,7.2250
26,3,NaN,0,0,7.2250
28,3,NaN,0,0,7.8792
29,3,NaN,0,0,7.8958
31,1,NaN,1,0,146.5208
32,3,NaN,0,0,7.7500
36,3,NaN,0,0,7.2292
42,3,NaN,0,0,7.8958


# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [0]:
"""
Your Code Here
"""

#補 Mean